In [2]:
import hana_ml
print(f"SAP HANA Client for Python: {hana_ml.__version__}")

SAP HANA Client for Python: 2.23.25010300


In [2]:
## HANA Trial
myhost='62f61f95-95b5-4ec0-a551-90d890eabbdf.hana.trial-us10.hanacloud.ondemand.com'
myport=443
myuser='DBADMIN'

In [3]:
## HANA Trial
myhost='b7c3ff95-9e0c-480d-9022-bfaaa268f780.hna0.prod-us10.hanacloud.ondemand.com'
myport=443
myuser='DBADMIN_VR'

In [4]:
from hana_ml import dataframe as hdf
myconn=hdf.ConnectionContext(
    address=myhost, 
    port=myport, 
    user=myuser
)
print(f"Connected to SAP HANA db version {myconn.hana_version()} \nat {myhost}:{myport} as user {myuser}")

Connected to SAP HANA db version 4.00.000.00.1736264738 (fa/CE2024.28) 
at b7c3ff95-9e0c-480d-9022-bfaaa268f780.hna0.prod-us10.hanacloud.ondemand.com:443 as user DBADMIN_VR


In [7]:
import pandas as pd
import webcolors

spec='css3'


In [ ]:

# Generate the list of color names and their RGB values
colors_data = [
    (wc, *webcolors.name_to_rgb(wc)) 
    for wc in webcolors.names(spec=spec)
]

# Create a DataFrame from the list
df = pd.DataFrame(colors_data, columns=["Color_Name", "Red", "Green", "Blue"])

# Display the DataFrame
print(df)


In [8]:
hdf_colors=hdf.create_dataframe_from_pandas(
    connection_context=myconn,
    table_name=f'COLORS_{spec}',
    pandas_df=df,
    force=True  # Overwrites the table if it already exists
)

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00, 53.70it/s]


In [10]:
hdf_colors=myconn.table(f'COLORS_{spec}')

In [11]:
hdf_colors.tail(10).collect()

,Color_Name,Red,Green,Blue
0,teal,0,128,128
1,thistle,216,191,216
2,tomato,255,99,71
3,turquoise,64,224,208
4,violet,238,130,238
5,wheat,245,222,179
6,white,255,255,255
7,whitesmoke,245,245,245
8,yellow,255,255,0
9,yellowgreen,154,205,50


In [12]:
(myconn
    .sql(f'''SELECT TOP 3 * FROM "COLORS_{spec}"''')
    .collect()
)

,Color_Name,Red,Green,Blue
0,aliceblue,240,248,255
1,antiquewhite,250,235,215
2,aqua,0,255,255


In [13]:
(myconn
    .sql(f'''
    WITH "SINGLE_COLOR" AS (SELECT TO_REAL_VECTOR(ARRAY("Red","Green","Blue")) AS "Ref_Color" FROM "COLORS_css3" WHERE "Color_Name"='navy')
SELECT TOP 1000
	"Color_Name",
	"Red",
	"Green",
	"Blue",
	ROUND(L2DISTANCE(TO_REAL_VECTOR(ARRAY("Red","Green","Blue")), "Ref_Color"), 2) AS "L2Dist",
	ROUND(COSINE_SIMILARITY(TO_REAL_VECTOR(ARRAY("Red","Green","Blue")), "Ref_Color"), 2) AS "CosSim"
FROM "COLORS_css3", "SINGLE_COLOR"
WHERE "Color_Name" <> 'black'
ORDER BY "CosSim" DESC, "L2Dist";
    ''')
    .collect()
)

,Color_Name,Red,Green,Blue,L2Dist,CosSim
0,navy,0,0,128,0.00,1.00
1,darkblue,0,0,139,11.00,1.00
2,mediumblue,0,0,205,77.00,1.00
3,blue,0,0,255,127.00,1.00
4,midnightblue,25,25,112,38.81,0.95
...,...,...,...,...,...,...
141,chartreuse,127,255,0,312.31,0.00
142,darkorange,255,140,0,317.82,0.00
143,orange,255,165,0,329.60,0.00
144,gold,255,215,0,357.26,0.00
